In [1]:
#IMPORT ALL THE REQUIRED LIBERARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
#  for warnings suppression
import warnings
warnings.filterwarnings('ignore')

# Set visualizations inline
%matplotlib inline

In [2]:
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
# Import the tqdm function from the tqdm library to display progress bars in loops
from tqdm import tqdm

In [5]:
chunks = pd.read_csv('INPUTEVENTS_CV.csv.gz', chunksize=100000)

#Concatenate first 3 chunks for inspection
df = pd.concat([chunk for i, chunk in enumerate(chunks) if i < 3])
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [6]:
df_sample = pd.read_csv('INPUTEVENTS_CV.csv.gz', nrows=5000)
df_sample.head()


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [7]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ROW_ID             5000 non-null   int64  
 1   SUBJECT_ID         5000 non-null   int64  
 2   HADM_ID            4977 non-null   float64
 3   ICUSTAY_ID         4965 non-null   float64
 4   CHARTTIME          5000 non-null   object 
 5   ITEMID             5000 non-null   int64  
 6   AMOUNT             4968 non-null   float64
 7   AMOUNTUOM          4962 non-null   object 
 8   RATE               0 non-null      float64
 9   RATEUOM            0 non-null      float64
 10  STORETIME          5000 non-null   object 
 11  CGID               5000 non-null   int64  
 12  ORDERID            5000 non-null   int64  
 13  LINKORDERID        5000 non-null   int64  
 14  STOPPED            39 non-null     object 
 15  NEWBOTTLE          7 non-null      float64
 16  ORIGINALAMOUNT     0 non

In [8]:
# Load the dataset (contains data from the iMDSoft MetaVision system)
#INPUTEVENTS_CV = pd.read_csv('INPUTEVENTS_CV.csv.gz')



In [9]:
# Preview the dataset
#INPUTEVENTS_CV.head()

In [10]:
#INPUTEVENTS_CV.info()

In [11]:
print(df_sample.shape)

(5000, 22)


In [12]:
columns_to_keep = [
    'SUBJECT_ID',     # Patient identifier
    'HADM_ID',        # Hospital admission ID
    'ICUSTAY_ID',     # ICU stay ID
    'CHARTTIME',      # Timestamp for event
    'ITEMID',         # Input item type
    'AMOUNT',         # Quantity administered
    'RATE'            # Rate of administration
]


In [13]:
 #Drop duplicates
df_sample.drop_duplicates(inplace=True)

# Convert CHARTTIME to datetime
df_sample['CHARTTIME'] = pd.to_datetime(df_sample['CHARTTIME'], errors='coerce')

# Drop columns with too many nulls or irrelevant for modeling
df_sample = df_sample.drop(columns=[
    'STOPPED','NEWBOTTLE','ORIGINALRATEUOM','ORIGINALSITE','ORIGINALROUTE','ORIGINALRATE','CGID','STORETIME','AMOUNTUOM',
    'STORETIME','LINKORDERID','AMOUNTUOM','RATEUOM','ORIGINALROUTE','CHARTTIME',
])

In [14]:
# Drop rows with missing critical fields
df_sample.dropna(subset=[ 'AMOUNT','ITEMID'], inplace=True)

In [15]:
#  Save cleaned data
df_sample.to_csv("INPUTEVENTS_CV_cleaned.csv", index=False)


In [16]:
!pip install sdv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

# Load cleaned dataset
inputevents_cv_cleaned = pd.read_csv("inputevents_cv_cleaned.csv")

# Generate metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df_sample)

# Initialize and train synthesizer
synthesizer = CTGANSynthesizer(metadata=metadata, epochs=20)
synthesizer.fit(inputevents_cv_cleaned)

# Sample synthetic data
synthetic = synthesizer.sample(num_rows=1000)
synthetic.to_csv("synthetic_inputevents_cv.csv", index=False)


InvalidDataError: The provided data does not match the metadata:
Errors in table:
Error: The columns ['CHARTTIME', 'RATEUOM'] are not present in the metadata.

In [ ]:
# Load real + synthetic for comparison
real = inputevents_cv_cleaned
synthetic = pd.read_csv("synthetic_inputevents_cv.csv")

# Plot distribution comparison
column = 'AMOUNT_scaled'
plt.figure(figsize=(10,5))
sns.histplot(real[column], color='blue', kde=True, label='Real', stat='density', bins=50, alpha=0.5)
sns.histplot(synthetic[column], color='red', kde=True, label='Synthetic', stat='density', bins=50, alpha=0.5)
plt.title(f'Real vs Synthetic: {column}')
plt.legend()
plt.show()


In [ ]:
# Example ML Evaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = ['ITEMID','AMOUNT_scaled']
target = 'AMOUNTUOM'  # Using a target column that has been encoded

X_real = real[features]
y_real = real[target]

Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_real, y_real, test_size=0.3)
model_real = RandomForestClassifier().fit(Xr_train, yr_train)
real_acc = accuracy_score(yr_test, model_real.predict(Xr_test))

X_syn = synthetic[features]
y_syn = synthetic[target]

Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_syn, y_syn, test_size=0.3)
model_syn = RandomForestClassifier().fit(Xs_train, ys_train)
syn_acc = accuracy_score(ys_test, model_syn.predict(Xs_test))

print(f"Real model accuracy:     {real_acc:.2f}")
print(f"Synthetic model accuracy: {syn_acc:.2f}")


In [ ]:
from sdv.single_table import TVAESynthesizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define features and target (from D_CPT structure)
features = ['ITEMID', 'AMOUNT_scaled']
target_col = AMOUNTUOM'

# Create binary classification target
inputevents_cv_cleaned['target'] = (inputevents_cv_cleaned[target_col] > inputevents_cv_cleaned[target_col].median()).astype(int)

# Regenerate metadata (if needed)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(inputevents_cv_cleaned.drop(columns=['target']))

# Fit TVAE (drop target column before fitting)
vae_synthesizer = TVAESynthesizer(metadata=metadata, epochs=10)
vae_synthesizer.fit(inputevents_cv_cleaned.drop(columns=['target']))

# Sample synthetic data
synthetic_data_vae = vae_synthesizer.sample(num_rows=1000)

# Add target to synthetic data
synthetic_data_vae['target'] = (synthetic_data_vae[target_col] > inputevents_cv_cleaned[target_col].median()).astype(int)

# Encode categorical features
for col in features:
    inputevents_cv_cleaned[col] = inputevents_cv_cleaned[col].astype('category').cat.codes
    synthetic_data_vae[col] = synthetic_data_vae[col].astype('category').cat.codes

# Train/test split
X_real = inputevents_cv_cleaned[features]
y_real = inputevents_cv_cleaned['target']
X_vae = synthetic_data_vae[features]
y_vae = synthetic_data_vae['target']

Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_real, y_real, test_size=0.3, random_state=42)
Xv_train, Xv_test, yv_train, yv_test = train_test_split(X_vae, y_vae, test_size=0.3, random_state=42)

# Train models
model_real = RandomForestClassifier(random_state=42)
model_real.fit(Xr_train, yr_train)
real_acc = accuracy_score(yr_test, model_real.predict(Xr_test))

model_vae = RandomForestClassifier(random_state=42)
model_vae.fit(Xv_train, yv_train)
vae_acc = accuracy_score(yv_test, model_vae.predict(Xv_test))

# Results
print(f"Model accuracy on REAL D_CPT data: {real_acc:.2f}")
print(f"Model accuracy on VAE SYNTHETIC D_CPT: {vae_acc:.2f}")


In [ ]:
import matplotlib.pyplot as plt

#  accuracy values
# These are from your previous results:
real_acc = real_acc          # Accuracy on real data using real training
vae_acc = vae_acc           # Accuracy on synthetic VAE data
gan_acc = syn_acc           # Accuracy on synthetic GAN (CTGAN) data

# Prepare bar chart data
labels = ['Real Data (Original)', 'CTGAN (Synthetic)', 'TVAE (Synthetic)']
accuracy_scores = [real_acc, gan_acc, vae_acc]

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(labels, accuracy_scores, color=['skyblue', 'orange', 'green'])

#  chart elements
plt.title('Model Accuracy Comparison: Real vs CTGAN vs TVAE', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy Score')
plt.ylim(0.2, 1.5)
plt.grid(axis='y', linestyle='--', alpha=0.4)

# Annotate bars
for i, acc in enumerate(accuracy_scores):
    plt.text(i, acc + 0.01, f"{acc:.2f}", ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()